In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Depression/GSE201332'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Peripheral whole blood: Healthy control and Major depression disorder patients"
!Series_summary	"Transcriptional profiling in the whole blood samples of healthy controls and major depression disorder (MDD) patients who have never been treated with depression medication. Samples included 20 healthy controls and 20 MDD patients who have never been treated with depression medication. Goal was to discover the differentially expressed genes."
!Series_overall_design	"20 Healthy controls vs. 20 major depression disorder (MDD) patients"
Sample Characteristics Dictionary:
{0: ['tissue: whole blood'], 1: ['subject status: heathy controls', 'subject status: MDD patients'], 2: ['gender: male', 'gender: female'], 3: ['age: 48y', 'age: 33y', 'age: 43y', 'age: 24y', 'age: 45y', 'age: 36y', 'age: 59y', 'age: 51y', 'age: 26y', 'age: 25y', 'age: 32y', 'age: 39y', 'age: 41y', 'age: 52y', 'age: 53y', 'age: 44y', 'age: 22y', 'age: 47y', 'age: 54y', 'age: 28y', 'age: 3

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking for gene expression data availability
is_gene_available = True  # inferred from the signals of gene expression in the background information

# Identifying available data and relevant keys
trait_row = 1 if len(set(sample_characteristics_dict[1])) > 1 else None  # subject status key
age_row = 3 if len(set(sample_characteristics_dict[3])) > 1 else None  # age key
gender_row = 2 if len(set(sample_characteristics_dict[2])) > 1 else None  # gender key

# Define conversion functions
def convert_trait(value):
    value = value.split(':')[-1].strip()
    if value == 'heathy controls':
        return 0
    elif value == 'MDD patients':
        return 1
    return None

def convert_age(value):
    try:
        return int(value.split(':')[-1].strip()[:-1])  # removing 'y' and converting to int
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(':')[-1].strip().lower()
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    return None

# Saving cohort information
save_cohort_info('GSE201332', './preprocessed/Depression/cohort_info.json', is_gene_available, trait_row is not None)

# Extracting and saving clinical features data if available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Depression', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Depression/trait_data/GSE201332.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM6058641': [0, 48, 1], 'GSM6058642': [0, 33, 1], 'GSM6058643': [0, 43, 1], 'GSM6058644': [0, 24, 1], 'GSM6058645': [0, 24, 1], 'GSM6058646': [0, 45, 1], 'GSM6058647': [0, 36, 1], 'GSM6058648': [0, 59, 1], 'GSM6058649': [0, 51, 1], 'GSM6058650': [0, 51, 1], 'GSM6058651': [0, 26, 0], 'GSM6058652': [0, 25, 0], 'GSM6058653': [0, 24, 0], 'GSM6058654': [0, 26, 0], 'GSM6058655': [0, 43, 0], 'GSM6058656': [0, 32, 0], 'GSM6058657': [0, 32, 0], 'GSM6058658': [0, 39, 0], 'GSM6058659': [0, 41, 0], 'GSM6058660': [0, 43, 0], 'GSM6058661': [1, 52, 1], 'GSM6058662': [1, 24, 1], 'GSM6058663': [1, 43, 1], 'GSM6058664': [1, 43, 0], 'GSM6058665': [1, 53, 0], 'GSM6058666': [1, 44, 0], 'GSM6058667': [1, 22, 0], 'GSM6058668': [1, 36, 0], 'GSM6058669': [1, 32, 1], 'GSM6058670': [1, 45, 1], 'GSM6058671': [1, 47, 1], 'GSM6058672': [1, 25, 1], 'GSM6058673': [1, 54, 1], 'GSM6058674': [1, 47, 0], 'GSM6058675': [1, 25, 0], 'GSM6058676': [1, 28, 0], 'GSM6058677': [1, 52, 0], 'GSM6058678': [1, 33, 0], 'GSM6058679

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['532', '532', '532', '532', '532'], 'ROW': [340.0, 338.0, 336.0, 334.0, 332.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'NAME'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function, and name the resulting gene expression dataframe 'gene_data'
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Depression/gene_data/GSE201332.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Depression')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE201332', './preprocessed/Depression/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Depression/GSE201332.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
A new JSON file was created at: ./preprocessed/Depression/cohort_info.json
